# 01 - LeeQ Basics: Introduction to Concepts and Simulation

This notebook provides an introduction to LeeQ concepts and demonstrates basic simulation capabilities.

## Learning Objectives
- Understand core LeeQ concepts
- Learn about quantum element abstractions
- Practice with simulation backends
- Explore Chronicle logging integration

## Prerequisites
- Basic understanding of quantum computing
- Python programming knowledge

## Setup and Imports

First, let's import the necessary LeeQ modules and set up our simulation environment.

In [ ]:
import leeq
import numpy as np
from leeq.chronicle import Chronicle, log_and_record
from leeq.core.elements.built_in.qudit_transmon import TransmonElement
from leeq.setups.built_in.setup_simulation_high_level import HighLevelSimulationSetup
from leeq.theory.simulation.numpy.rotated_frame_simulator import VirtualTransmon
from leeq.experiments.experiments import ExperimentManager

# Import necessary visualization tools
import plotly.graph_objects as go
import plotly.express as px

print("LeeQ imported successfully!")

# Core LeeQ imports explanation:
# - TransmonElement: Represents a superconducting transmon qubit 
# - HighLevelSimulationSetup: Simulation backend for quantum experiments
# - VirtualTransmon: Simulated transmon with realistic parameters
# - Chronicle: Experiment logging and data persistence system
# - ExperimentManager: Manages experimental setups and configurations

## Chronicle Integration

LeeQ uses Chronicle for experiment logging and data persistence.

In [ ]:
# Start Chronicle logging
Chronicle().start_log()

# Chronicle is LeeQ's logging and data persistence system
print("Chronicle logging started successfully!")

# Chronicle provides comprehensive experiment logging
print("Chronicle features:")
print("- Automatic logging of experiment parameters and results")
print("- Data persistence across experimental sessions")
print("- Hierarchical organization of experimental data")
print("- Integration with LeeQ experimental framework")

# Chronicle automatically logs:
# - Function calls and parameters (for LoggableObject methods)
# - Experiment results and metadata
# - Configuration changes
# - Error messages and debugging information

## Core LeeQ Concepts

### Quantum Elements
LeeQ uses quantum elements to represent physical qubits and their properties. The **TransmonElement** is the most common element, representing a superconducting transmon qubit.

Key properties of a transmon qubit:
- **Frequency**: The transition frequency between qubit levels (typically 4-6 GHz)
- **Anharmonicity**: The difference between transition frequencies (typically -200 MHz)
- **Coherence times**: T1 (relaxation) and T2 (dephasing) times
- **Readout frequency**: The frequency used to measure the qubit state

### Simulation Backends  
LeeQ provides multiple simulation backends for quantum experiments:
- **HighLevelSimulationSetup**: High-level simulation for educational and testing purposes
- **VirtualTransmon**: Realistic transmon simulation with noise and decoherence
- **Hardware backends**: Interface to actual quantum devices (not covered in this notebook)

### Experiment Framework
LeeQ experiments follow a consistent pattern:
1. **Setup**: Configure the experimental environment
2. **Calibration**: Determine optimal parameters
3. **Execution**: Run the actual experiment  
4. **Analysis**: Process and visualize results

In [ ]:
# Create a virtual transmon with realistic parameters
virtual_transmon = VirtualTransmon(
    name="BasicQubit",
    qubit_frequency=5040.0,    # 5.04 GHz qubit frequency
    anharmonicity=-198.0,      # -198 MHz anharmonicity 
    t1=70.0,                   # 70 μs T1 relaxation time
    t2=35.0,                   # 35 μs T2 dephasing time
    readout_frequency=9645.0,  # 9.645 GHz readout frequency
    quiescent_state_distribution=np.array([0.8, 0.15, 0.04, 0.01])  # Population in states |0⟩, |1⟩, |2⟩, |3⟩
)

print("Virtual Transmon Properties:")
print(f"- Qubit frequency: {virtual_transmon.qubit_frequency:.1f} MHz")
print(f"- Anharmonicity: {virtual_transmon.anharmonicity:.1f} MHz")
print(f"- T1 relaxation: {virtual_transmon.t1:.1f} μs")
print(f"- T2 dephasing: {virtual_transmon.t2:.1f} μs")
print(f"- Readout frequency: {virtual_transmon.readout_frequency:.1f} MHz")

# Create the simulation setup
manager = ExperimentManager()
manager.clear_setups()

setup = HighLevelSimulationSetup(
    name='BasicSimulationSetup',
    virtual_qubits={1: virtual_transmon},  # Assign to channel 1
)

manager.register_setup(setup)
print(f"\\nSimulation setup '{setup.name}' created with 1 virtual qubit")

# Configure the TransmonElement with realistic parameters
qubit_config = {
    'hrid': 'BasicQubit',
    'lpb_collections': {
        'f01': {  # |0⟩ → |1⟩ transition
            'type': 'SimpleDriveCollection',
            'freq': 5040.0,
            'channel': 1,
            'shape': 'blackman_drag',
            'amp': 0.5,
            'phase': 0.0,
            'width': 0.05,
            'alpha': 500,
            'trunc': 1.2
        },
    },
    'measurement_primitives': {
        '0': {  # Readout measurement
            'type': 'SimpleDispersiveMeasurement', 
            'freq': 9645.0,
            'channel': 1,
            'shape': 'square',
            'amp': 0.15,
            'phase': 0.0,
            'width': 1.0,
            'trunc': 1.2,
            'distinguishable_states': [0, 1]
        }
    }
}

# Create the TransmonElement
qubit = TransmonElement(name=qubit_config['hrid'], parameters=qubit_config)
print(f"\\nTransmonElement created with HRID: {qubit_config['hrid']}")
# Print basic configuration information
try:
    qubit.print_config_info()
except Exception as e:
    print(f"Configuration display not available in this environment: {e}")
    print("TransmonElement configured successfully!")

## Basic Measurements and Data Visualization

Now let's demonstrate basic qubit measurements and create visualizations to understand quantum behavior.

In [ ]:
# Demonstrate basic qubit operations and measurements
from leeq.experiments.builtin.basic.characterizations import SimpleT1

# Set up experimental parameters
setup.status.set_param("Shot_Number", 1000)  # Number of measurement shots
setup.status.set_param("Shot_Period", 500)   # Time between shots (μs)

print("Basic Qubit State Information:")
print(f"Ground state |0⟩ population: {virtual_transmon.quiescent_state_distribution[0]:.1%}")
print(f"Excited state |1⟩ population: {virtual_transmon.quiescent_state_distribution[1]:.1%}")
print(f"Higher excited states: {virtual_transmon.quiescent_state_distribution[2:]}")

# Get qubit control parameters
c1_pulse = qubit.get_c1('f01')  # Control pulse for |0⟩ → |1⟩ transition
readout_params = qubit.get_measurement_prim_intlist(0)  # Readout measurement parameters

print(f"\\nQubit Control Parameters:")
print(f"- Control pulse collection: {type(c1_pulse).__name__}")
print(f"- Configuration from qubit_config: freq={qubit_config['lpb_collections']['f01']['freq']:.1f} MHz")
print(f"- Pulse amplitude: {qubit_config['lpb_collections']['f01']['amp']:.3f}")
print(f"- Pulse width: {qubit_config['lpb_collections']['f01']['width']:.3f} μs")
print(f"- Pulse shape: {qubit_config['lpb_collections']['f01']['shape']}")

print(f"\\nReadout Parameters:")
print(f"- Readout primitive: {type(readout_params).__name__}")
print(f"- Configuration from qubit_config: freq={qubit_config['measurement_primitives']['0']['freq']:.1f} MHz") 
print(f"- Readout amplitude: {qubit_config['measurement_primitives']['0']['amp']:.3f}")
print(f"- Readout duration: {qubit_config['measurement_primitives']['0']['width']:.1f} μs")

In [ ]:
# Perform a basic T1 relaxation measurement
print("Running T1 relaxation measurement...")
print("This measures how long the qubit stays in the excited state before relaxing to ground state.")

try:
    # Run T1 experiment with short time range for quick demonstration
    t1_experiment = SimpleT1(
        qubit=qubit,
        time_length=50,    # Maximum delay time (μs)
        time_resolution=2  # Time step (μs)  
    )
    
    # The experiment automatically fits the data and extracts T1
    print(f"\\nT1 Measurement Complete!")
    print(f"Measured T1 relaxation time: {getattr(t1_experiment, 'fitted_t1', 'N/A')} μs")
    
except Exception as e:
    print(f"T1 experiment encountered an issue: {e}")
    print("This is normal in simulation environments - continuing with demonstration.")

In [ ]:
# Create visualizations to understand quantum concepts
fig_concepts = go.Figure()

# Plot qubit energy levels
energy_levels = [0, virtual_transmon.qubit_frequency, 
                virtual_transmon.qubit_frequency + virtual_transmon.qubit_frequency + virtual_transmon.anharmonicity]

fig_concepts.add_trace(go.Scatter(
    x=[0, 1, 2], 
    y=energy_levels[:3],
    mode='lines+markers+text',
    text=['|0⟩', '|1⟩', '|2⟩'],
    textposition='middle right',
    name='Energy Levels',
    line=dict(color='blue', width=3),
    marker=dict(size=12)
))

fig_concepts.add_annotation(
    x=0.5, y=virtual_transmon.qubit_frequency/2,
    text=f'f₀₁ = {virtual_transmon.qubit_frequency:.0f} MHz',
    showarrow=True, arrowhead=2, arrowcolor='red'
)

fig_concepts.add_annotation(
    x=1.5, y=virtual_transmon.qubit_frequency + virtual_transmon.anharmonicity/2,
    text=f'f₁₂ = f₀₁ + α\\nα = {virtual_transmon.anharmonicity:.0f} MHz',
    showarrow=True, arrowhead=2, arrowcolor='red'
)

fig_concepts.update_layout(
    title='Transmon Qubit Energy Level Structure',
    xaxis_title='Qubit Level',
    yaxis_title='Energy (MHz)',
    showlegend=True,
    width=700, height=500
)

fig_concepts.show()

# Show state population distribution
fig_pop = go.Figure(data=go.Bar(
    x=['|0⟩', '|1⟩', '|2⟩', '|3⟩'],
    y=virtual_transmon.quiescent_state_distribution,
    marker_color=['lightblue', 'lightcoral', 'lightgreen', 'lightyellow']
))

fig_pop.update_layout(
    title='Qubit State Population Distribution',
    xaxis_title='Quantum State',
    yaxis_title='Population',
    width=600, height=400
)

fig_pop.show()

print("\\n📊 The visualizations above show:")
print("1. Energy level structure: Higher levels are closer together (anharmonicity)")
print("2. State populations: Mostly in ground state |0⟩, small thermal populations in excited states")

## Key Concepts Summary

### What We've Learned

1. **LeeQ Architecture**: LeeQ provides a comprehensive framework for quantum experiments
   - **Chronicle**: Automatic logging and data persistence
   - **Elements**: Quantum device abstractions (TransmonElement)
   - **Setups**: Simulation and hardware backends
   - **Experiments**: Structured experimental procedures

2. **Transmon Qubits**: 
   - **Weakly anharmonic oscillators** with energy levels that get closer together
   - **Controllable** using microwave pulses at specific frequencies
   - **Measurable** using dispersive readout techniques
   - **Decoherent** due to T1 (relaxation) and T2 (dephasing) processes

3. **Simulation Framework**:
   - **VirtualTransmon**: Realistic qubit simulation with noise and thermal populations
   - **HighLevelSimulationSetup**: Educational simulation backend
   - **Configurable parameters**: Frequencies, coherence times, pulse shapes

4. **Data and Visualization**:
   - **Automatic data logging** through Chronicle integration
   - **Interactive plots** using Plotly for better understanding
   - **Experimental analysis** with built-in fitting procedures

## Next Steps

Continue to [02_single_qubit.ipynb](02_single_qubit.ipynb) to learn about single qubit experiments and calibration.